# Playing with OWLready (`owlready2`)

In [4]:
# Install owlready2
# !pip install owlready2

In [5]:
# Import essential modules
from owlready2 import *
import datetime

# Your First OWL Ontology

In [6]:
# Create a new ontology
onto = get_ontology("http://example.org/music.owl")

In [39]:
# Define classes using Python class syntax
with onto:
    class Song(Thing):
        pass
    
    class Artist(Thing):
        pass
    
    # Define properties as Python properties
    class performedBy(ObjectProperty):
        domain = [Song]
        range = [Artist]
    
    class title(DataProperty):
        domain = [Song]
        range = [str]

    class title(AnnotationProperty):
        pass
    
    class comment(AnnotationProperty):
        pass

    

In [40]:
# Create instances
bohemian_rhapsody = Song("BohemianRhapsody")
queen = Artist("Queen")

In [41]:
# Set properties using Python assignment
bohemian_rhapsody.title = ["Bohemian Rhapsody"]
bohemian_rhapsody.performedBy = [queen]

In [42]:
print(f"Song: {bohemian_rhapsody.title[0]}")
print(f"Performed by: {bohemian_rhapsody.performedBy[0]}")

Song: Bohemian Rhapsody
Performed by: music.Queen


In [43]:
# dir(onto)

In [44]:
# dir(bohemian_rhapsody)

# Namespaces

In [ ]:
# Set ontology metadata
onto.metadata.title = ["Music Ontology"]
onto.metadata.comment = ["An ontology for music domain modeling"]

# Access the ontology's namespace
print(f"Ontology IRI: {onto.base_iri}")
print(f"All classes: {list(onto.classes())}")
print(f"All properties: {list(onto.properties())}")

In [48]:
# You can also work with imported ontologies
# foaf = get_ontology("http://xmlns.com/foaf/0.1/").load()
# Load FOAF ontology from web
try:
    foaf = get_ontology("http://xmlns.com/foaf/0.1/").load()
    print("FOAF ontology loaded successfully")
    
    # Explore FOAF classes
    print("FOAF classes:")
    for cls in foaf.classes():
        print(f"  {cls.name}")
    
    # Explore FOAF properties  
    print("FOAF properties:")
    for prop in foaf.properties():
        print(f"  {prop.name}")
        
except Exception as e:
    print(f"Failed to load FOAF: {e}")
    # Define minimal FOAF classes locally
    with onto:
        class Person(Thing): pass
        class name(DataProperty): 
            domain = [Person]
            range = [str]
foaf

FOAF ontology loaded successfully
FOAF classes:
  Class
  Person
  Person
  Agent
  Agent
  Person
  SpatialThing
  Document
  Organization
  Project
  Document
  Organization
  Group
  Agent-3
  Project
  Image
  PersonalProfileDocument
  OnlineAccount
  OnlineGamingAccount
  OnlineEcommerceAccount
  OnlineChatAccount
FOAF properties:
  mbox_sha1sum
  gender
  geekcode
  dnaChecksum
  sha1
  title
  nick
  jabberID
  aimChatID
  icqChatID
  yahooChatID
  msnChatID
  name
  firstName
  givenname
  surname
  family_name
  plan
  accountName
  birthday
  mbox
  based_near
  phone
  homepage
  page
  weblog
  tipjar
  made
  maker
  img
  depiction
  depicts
  thumbnail
  myersBriggs
  workplaceHomepage
  workInfoHomepage
  schoolHomepage
  knows
  interest
  topic_interest
  publications
  currentProject
  pastProject
  fundedBy
  logo
  topic
  primaryTopic
  theme
  holdsAccount
  accountServiceHomepage
  member
  assurance
  src_assurance
  term_status
  description
  title
  date
  m

get_ontology("http://xmlns.com/foaf/0.1/")

In [ ]:
# Use FOAF in music ontology
with onto:
    # Extend Artist to be a FOAF Person
    class Artist(foaf.Person):  # Inherits from FOAF Person
        pass
    
    class bandMember(ObjectProperty):
        domain = [Artist] 
        range = [foaf.Person]  # Can reference any FOAF Person

# Create instances using FOAF properties
freddie = Artist("FreddieMe")
freddie.foaf.name = ["Freddie Mercury"]  # Use FOAF name property
freddie.name = ["Freddie Mercury"]       # Or music ontology name property

print(f"Artist created: {freddie.foaf.name[0]}")

In [38]:
for i in onto.metadata:
    print(i)
dir(onto.metadata)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'namespace',
 'storid']

# Loading External RDF/OWL Data

In [18]:
# Load the existing music ontology
music_onto = get_ontology("./rdf_output/music_ontology.rdf").load()

In [19]:
# Explore loaded classes
print("Classes in the ontology:")
for cls in music_onto.classes():
    print(f"  {cls.name}: {cls.comment}")

Classes in the ontology:
  Artist: ['A person or group who creates or performs music']
  CollaborativeSong: ['A song performed by multiple artists']
  Song: ['A musical composition']
  Award: ['A recognition given for musical achievement']
  Album: ['A collection of songs released together']
  Genre: ['A category or style of music']
  Single: ['A song released as a standalone track or as a primary track from an album']
  SuccessfulLabel: ['A record label with multiple award-winning artists']
  RecordLabel: ['A company that manages the production, distribution, and promotion of music']
  ExtendedPlay: ['A musical recording that contains more music than a single but is shorter than an album']
  EstablishedArtist: ['An artist with multiple albums and awards']


In [20]:
# Explore properties
print("\nProperties in the ontology:")
for prop in music_onto.properties():
    print(f"  {prop.name}: domain={prop.domain}, range={prop.range}")


Properties in the ontology:
  popularityScore: domain=[], range=[<class 'int'>]
  nationality: domain=[], range=[<class 'str'>]
  description: domain=[], range=[<class 'str'>]
  location: domain=[], range=[<class 'str'>]
  labelName: domain=[], range=[<class 'str'>]
  title: domain=[], range=[<class 'str'>]
  birthDate: domain=[], range=[<class 'datetime.date'>]
  awardName: domain=[], range=[<class 'str'>]
  collaborationStrength: domain=[], range=[<class 'int'>]
  name: domain=[], range=[<class 'str'>]
  awardingBody: domain=[], range=[<class 'str'>]
  releaseYear: domain=[], range=[<class 'int'>]
  labelSuccessRating: domain=[], range=[<class 'int'>]
  albumTitle: domain=[], range=[<class 'str'>]
  year: domain=[], range=[<class 'int'>]
  duration: domain=[], range=[<class 'int'>]
  genreName: domain=[], range=[<class 'str'>]
  releaseDate: domain=[], range=[<class 'datetime.date'>]
  collaboratesWith: domain=[], range=[]
  influencedBy: domain=[], range=[]
  performedBy: domain=[]

In [21]:
# Check reasoner-inferred classes
print(f"\nTotal classes: {len(list(music_onto.classes()))}")
print(f"Total properties: {len(list(music_onto.properties()))}")


Total classes: 11
Total properties: 29


# Basic Queries

owlready2 offers two query approaches.   
* Python iteration and comprehensions work well for simple queries.
* SPARQL handles complex aggregations that would be harder to maintain in pure Python.

In [22]:
# Helper function
def format_sparql_results(results, query_description = ""):
    print(f"\n{query_description}")
    print("=" * len(query_description))
    # 
    results_list = list(results)
    if not results_list:
        print("No results found.")
        return
    # 
    # Get variable names from first result
    if hasattr(results_list[0], '__dict__'):
        # print('__dict__ found in results')
        headers = list(results_list[0].__dict__.keys())
    else:
        headers = [f"col_{i}" for i in range(len(results_list[0]))]
    # print ('headers = ',headers)
    # 
    # Format headers
    print(" | ".join(f"{h:20}" for h in headers))
    print("-" * (23 * len(headers) - 3))
    
    # Format rows
    for row in results_list[:10]:  # Limit to first 10 results
        formatted_row = []
        for item in row:
            if item is None:
                formatted_row.append("None")
            elif hasattr(item, 'split') and '#' in str(item):
                # Extract local name from URI
                formatted_row.append(str(item).split('#')[-1])
            elif hasattr(item, 'split') and '/' in str(item):
                # Extract last part of path
                formatted_row.append(str(item).split('/')[-1])
            else:
                formatted_row.append(str(item))
        print(" | ".join(f"{item:20}" for item in formatted_row))
    
    if len(results_list) > 10:
        print(f"... and {len(results_list) - 10} more results")


# Usage:
# format_sparql_results(results, description)

In [27]:
# Query 1: Find all songs and their performers using Python iteration
print("Query 1: Songs and performers")
out = []
for song in music_onto.Song.instances():
    if hasattr(song, 'performedBy') and song.performedBy:
        for performer in song.performedBy:
            out.append(f"{song.name} performed by {performer.name}")
out[:10]

Query 1: Songs and performers


['song_song_421 performed by artist_artist_35',
 'song_song_421 performed by artist_artist_20',
 'song_song_386 performed by artist_artist_20',
 'song_song_386 performed by artist_artist_50',
 'song_song_541 performed by artist_artist_19',
 'song_song_541 performed by artist_artist_56',
 'song_song_244 performed by artist_artist_23',
 'song_song_244 performed by artist_artist_25',
 'song_song_11 performed by artist_artist_90',
 'song_song_11 performed by artist_artist_75']

In [28]:
# Query 2: Find artists signed to labels using property filtering
print("\nQuery 2: Artists and their labels")
out = []
for artist in music_onto.Artist.instances():
    if hasattr(artist, 'signedTo') and artist.signedTo:
        out.append(f"{artist.name} signed to {artist.signedTo[0].name}")
out[:10]


Query 2: Artists and their labels


[]

In [ ]:
# Query 3: Use SPARQL for complex aggregation queries
print("\nQuery 3: Songs by genre (using SPARQL)")
sparql_query = """
SELECT ?genre (COUNT(?song) as ?count) WHERE {
    ?song rdf:type music:Song .
    ?song music:hasGenre ?genre .
}
GROUP BY ?genre
ORDER BY DESC(?count)
"""
# owlready2 supports SPARQL through the default_world
results = default_world.sparql(sparql_query)
for genre, count in results:
    print(f"  Genre: {genre}, Songs: {count}")

In [32]:
# Query 4: Find collaborative songs using list comprehension
print("\nQuery 4: Collaborative songs")
collaborative_songs = [song for song in music_onto.Song.instances() 
                      if hasattr(song, 'performedBy') and len(song.performedBy) > 1]
out = []
for song in collaborative_songs:
    performers = [p.name for p in song.performedBy]
    out.append(f"  {song.name}: {', '.join(performers)}")
out[:10]


Query 4: Collaborative songs


['  song_song_421: artist_artist_35, artist_artist_20',
 '  song_song_386: artist_artist_20, artist_artist_50',
 '  song_song_541: artist_artist_19, artist_artist_56',
 '  song_song_244: artist_artist_23, artist_artist_25',
 '  song_song_11: artist_artist_90, artist_artist_75',
 '  song_song_419: artist_artist_56, artist_artist_58',
 '  song_song_316: artist_artist_83, artist_artist_22',
 '  song_song_319: artist_artist_85, artist_artist_77',
 '  song_song_404: artist_artist_29, artist_artist_84',
 '  song_song_60: artist_artist_26, artist_artist_57']

In [33]:
# Query 5: Albums and track counts using Python aggregation
print("\nQuery 5: Albums and track counts")
album_tracks = {}
for song in music_onto.Song.instances():
    if hasattr(song, 'featuredOn'):
        for album in song.featuredOn:
            album_name = album.name if hasattr(album, 'name') else str(album)
            album_tracks[album_name] = album_tracks.get(album_name, 0) + 1

for album, count in album_tracks.items():
    print(f"  {album}: {count} tracks")


Query 5: Albums and track counts


# Creating Core Music Classes and Properties

In [24]:
# Create a new graph for our ontology extension
g = Graph()
music = Namespace("http://example.org/music#")
g.bind("music", music)

## Define core classes

In [25]:
g.add((music.Playlist, RDF.type, OWL.Class))
g.add((music.Playlist, RDFS.label, Literal("Playlist")))

<Graph identifier=N44d6198f23be46d0b85b7647484f08cb (<class 'rdflib.graph.Graph'>)>

In [26]:
g.add((music.Concert, RDF.type, OWL.Class))
g.add((music.Concert, RDFS.label, Literal("Concert")))

<Graph identifier=N44d6198f23be46d0b85b7647484f08cb (<class 'rdflib.graph.Graph'>)>

In [27]:
g.add((music.MusicVideo, RDF.type, OWL.Class))
g.add((music.MusicVideo, RDFS.label, Literal("Music Video")))

<Graph identifier=N44d6198f23be46d0b85b7647484f08cb (<class 'rdflib.graph.Graph'>)>

## Define some properties for Playlist

In [28]:
g.add((music.containsSong, RDF.type, OWL.ObjectProperty))
g.add((music.containsSong, RDFS.domain, music.Playlist))
g.add((music.containsSong, RDFS.range, music.Song))

<Graph identifier=N44d6198f23be46d0b85b7647484f08cb (<class 'rdflib.graph.Graph'>)>

In [29]:
g.add((music.playlistName, RDF.type, OWL.DatatypeProperty))
g.add((music.playlistName, RDFS.domain, music.Playlist))
g.add((music.playlistName, RDFS.range, XSD.string))

<Graph identifier=N44d6198f23be46d0b85b7647484f08cb (<class 'rdflib.graph.Graph'>)>

In [30]:
g.add((music.createdBy, RDF.type, OWL.ObjectProperty))
g.add((music.createdBy, RDFS.domain, music.Playlist))
g.add((music.createdBy, RDFS.range, FOAF.Person))

<Graph identifier=N44d6198f23be46d0b85b7647484f08cb (<class 'rdflib.graph.Graph'>)>

In [31]:
# Define properties for Concert
g.add((music.performsAt, RDF.type, OWL.ObjectProperty))
g.add((music.performsAt, RDFS.domain, music.Artist))
g.add((music.performsAt, RDFS.range, music.Concert))

<Graph identifier=N44d6198f23be46d0b85b7647484f08cb (<class 'rdflib.graph.Graph'>)>

In [32]:
g.add((music.concertDate, RDF.type, OWL.DatatypeProperty))
g.add((music.concertDate, RDFS.domain, music.Concert))
g.add((music.concertDate, RDFS.range, XSD.date))

<Graph identifier=N44d6198f23be46d0b85b7647484f08cb (<class 'rdflib.graph.Graph'>)>

In [33]:
g.add((music.venue, RDF.type, OWL.DatatypeProperty))
g.add((music.venue, RDFS.domain, music.Concert))
g.add((music.venue, RDFS.range, XSD.string))

<Graph identifier=N44d6198f23be46d0b85b7647484f08cb (<class 'rdflib.graph.Graph'>)>

## Define some properties for MusicVideo

In [34]:
g.add((music.videoFor, RDF.type, OWL.ObjectProperty))
g.add((music.videoFor, RDFS.domain, music.MusicVideo))
g.add((music.videoFor, RDFS.range, music.Song))

<Graph identifier=N44d6198f23be46d0b85b7647484f08cb (<class 'rdflib.graph.Graph'>)>

In [35]:
g.add((music.director, RDF.type, OWL.DatatypeProperty))
g.add((music.director, RDFS.domain, music.MusicVideo))
g.add((music.director, RDFS.range, XSD.string))

<Graph identifier=N44d6198f23be46d0b85b7647484f08cb (<class 'rdflib.graph.Graph'>)>

In [36]:
g.add((music.viewCount, RDF.type, OWL.DatatypeProperty))
g.add((music.viewCount, RDFS.domain, music.MusicVideo))
g.add((music.viewCount, RDFS.range, XSD.int))

<Graph identifier=N44d6198f23be46d0b85b7647484f08cb (<class 'rdflib.graph.Graph'>)>

# Working with Individuals

In [37]:
# Create specific instances
my_playlist = URIRef("http://example.org/playlist/roadtrip2024")
queen_concert = URIRef("http://example.org/concert/wembley1986")
bohemian_video = URIRef("http://example.org/video/bohemian_rhapsody")

In [38]:
# Add instance data for playlist
g.add((my_playlist, RDF.type, music.Playlist))
g.add((my_playlist, music.playlistName, Literal("Road Trip 2024")))
g.add((my_playlist, music.containsSong, music.BohemianRhapsody))

<Graph identifier=N44d6198f23be46d0b85b7647484f08cb (<class 'rdflib.graph.Graph'>)>

In [39]:
# Add instance data for concert
g.add((queen_concert, RDF.type, music.Concert))
g.add((queen_concert, music.concertDate, Literal("1986-07-12", datatype=XSD.date)))
g.add((queen_concert, music.venue, Literal("Wembley Stadium")))
g.add((music.Queen, music.performsAt, queen_concert))

<Graph identifier=N44d6198f23be46d0b85b7647484f08cb (<class 'rdflib.graph.Graph'>)>

In [40]:
# Add instance data for music video
g.add((bohemian_video, RDF.type, music.MusicVideo))
g.add((bohemian_video, music.videoFor, music.BohemianRhapsody))
g.add((bohemian_video, music.director, Literal("Bruce Gowers")))
g.add((bohemian_video, music.viewCount, Literal(1500000000, datatype=XSD.int)))

<Graph identifier=N44d6198f23be46d0b85b7647484f08cb (<class 'rdflib.graph.Graph'>)>

# Possible Restrictions for Music Domain

In [41]:
# Create restriction: albums that contain rock songs
rock_album_restriction = BNode()

In [42]:
g.add((rock_album_restriction, RDF.type, OWL.Restriction))
g.add((rock_album_restriction, OWL.onProperty, music.features))
g.add((rock_album_restriction, OWL.someValuesFrom, music.RockSong))

<Graph identifier=N44d6198f23be46d0b85b7647484f08cb (<class 'rdflib.graph.Graph'>)>

In [43]:
# Define RockAlbum as albums with rock content
g.add((music.RockAlbum, RDF.type, OWL.Class))
g.add((music.RockAlbum, RDFS.subClassOf, music.Album))
g.add((music.RockAlbum, RDFS.subClassOf, rock_album_restriction))

<Graph identifier=N44d6198f23be46d0b85b7647484f08cb (<class 'rdflib.graph.Graph'>)>

In [44]:
# Create cardinality restriction: singles have exactly one song
single_restriction = BNode()
g.add((single_restriction, RDF.type, OWL.Restriction))
g.add((single_restriction, OWL.onProperty, music.features))
g.add((single_restriction, OWL.cardinality, Literal(1, datatype=XSD.nonNegativeInteger)))

<Graph identifier=N44d6198f23be46d0b85b7647484f08cb (<class 'rdflib.graph.Graph'>)>

In [45]:
g.add((music.Single, RDFS.subClassOf, single_restriction))

<Graph identifier=N44d6198f23be46d0b85b7647484f08cb (<class 'rdflib.graph.Graph'>)>

In [46]:
# Create domain/range restriction for award relationships
g.add((music.hasWonAward, RDFS.domain, music.Artist))
g.add((music.hasWonAward, RDFS.range, music.Award))

<Graph identifier=N44d6198f23be46d0b85b7647484f08cb (<class 'rdflib.graph.Graph'>)>

# Basic Inference with Music Data

In [47]:
# rdflib has limited built-in OWL reasoning
# We can implement simple RDFS inference manually

def apply_rdfs_inference(graph):
    """Apply basic RDFS subclass inference"""
    inferred = Graph()
    inferred += graph  # Copy original
    
    # Apply subclass transitivity
    changed = True
    while changed:
        changed = False
        for s, p, o in inferred.triples((None, RDFS.subClassOf, None)):
            for s2, p2, o2 in inferred.triples((None, RDFS.subClassOf, s)):
                triple = (s2, RDFS.subClassOf, o)
                if triple not in inferred:
                    inferred.add(triple)
                    changed = True
    
    # Apply type inference from subclasses
    for individual, rdf_type, cls in inferred.triples((None, RDF.type, None)):
        for subcls, subclass_of, supercls in inferred.triples((cls, RDFS.subClassOf, None)):
            triple = (individual, RDF.type, supercls)
            if triple not in inferred:
                inferred.add(triple)
    
    return inferred

In [48]:
# Apply inference
g_inferred = apply_rdfs_inference(g)

In [49]:
# Query with inference
query = """
SELECT ?individual ?type WHERE {
    ?individual rdf:type ?type .
    ?type rdfs:subClassOf music:Album .
}
"""

In [50]:
results = g_inferred.query(query, initNs={"music": music, "rdf": RDF, "rdfs": RDFS})
print("Individuals that are albums (including subtypes):")
for row in results:
    print(f"  {row.individual} is a {row.type}")

Individuals that are albums (including subtypes):


# Serializing Music Ontology Data

In [51]:
# Serialize in different formats for different use cases
g.serialize(destination="toy_music.ttl", format="turtle")
g.serialize(destination="toy_music.owl", format="xml")
g.serialize(destination="toy_music.jsonld", format="json-ld")

<Graph identifier=N44d6198f23be46d0b85b7647484f08cb (<class 'rdflib.graph.Graph'>)>

In [52]:
# Print turtle format (most human-readable)
print("Turtle serialization:")
print(g.serialize(format="turtle"))

Turtle serialization:
@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix music: <http://example.org/music#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

music:Queen music:performsAt <http://example.org/concert/wembley1986> .

music:RockAlbum a owl:Class ;
    rdfs:subClassOf [ a owl:Restriction ;
            owl:onProperty music:features ;
            owl:someValuesFrom music:RockSong ],
        music:Album .

music:Single rdfs:subClassOf [ a owl:Restriction ;
            owl:cardinality "1"^^xsd:nonNegativeInteger ;
            owl:onProperty music:features ] .

music:concertDate a owl:DatatypeProperty ;
    rdfs:domain music:Concert ;
    rdfs:range xsd:date .

music:containsSong a owl:ObjectProperty ;
    rdfs:domain music:Playlist ;
    rdfs:range music:Song .

music:createdBy a owl:ObjectProperty ;
    rdfs:domain music:Playlist ;
    rdfs:range foaf:Person .

music:d

In [53]:
# Save only the new classes we created
new_classes_graph = Graph()
new_classes_graph.bind("music", music)

In [54]:
for s, p, o in g.triples((None, RDF.type, OWL.Class)):
    if str(s).startswith("http://example.org/music#"):
        # Add all triples about this class
        for s2, p2, o2 in g.triples((s, None, None)):
            new_classes_graph.add((s2, p2, o2))

In [55]:
new_classes_graph.serialize(destination="toy_new_music_classes.ttl", format="turtle")

<Graph identifier=N5d0e54eab3784e9db47b7b134f64c6fa (<class 'rdflib.graph.Graph'>)>

Different serialization formats serve different purposes. 
* Turtle is human-readable for debugging
* XML integrates with OWL tools
* JSON-LD works with web applications.
* Separating new classes helps with modular ontology development.

.